In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.abspath('')
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [49]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import re
import pickle

import importlib
import utils
importlib.reload(utils)
from tqdm import tqdm

from utils import get_parsed, get_parsed_remove_comment, baseball_base_url, ordinal, clean_name

In [9]:
df_all_leaders = pd.read_csv('./all_leaders.csv')

In [10]:
df_all_leaders.head(3)

,Statistic,Single-Season,Career,Active,Progressive,Yearly League,Year-by-Year Top-Tens,Category
0,Wins Above Replacement,/leaders/WAR_season.shtml,/leaders/WAR_career.shtml,/leaders/WAR_active.shtml,/leaders/WAR_progress.shtml,/leaders/WAR_leagues.shtml,/leaders/WAR_top_ten.shtml,Player Value Leaderboards
1,WAR Position Players,/leaders/WAR_bat_season.shtml,/leaders/WAR_bat_career.shtml,/leaders/WAR_bat_active.shtml,/leaders/WAR_bat_progress.shtml,/leaders/WAR_bat_leagues.shtml,/leaders/WAR_bat_top_ten.shtml,Batting Leaderboards
2,Offensive WAR,/leaders/WAR_off_season.shtml,/leaders/WAR_off_career.shtml,/leaders/WAR_off_active.shtml,/leaders/WAR_off_progress.shtml,/leaders/WAR_off_leagues.shtml,/leaders/WAR_off_top_ten.shtml,Batting Leaderboards


In [11]:
df_active = df_all_leaders[['Statistic', 'Active', 'Category']]
df_active.head(2)

,Statistic,Active,Category
0,Wins Above Replacement,/leaders/WAR_active.shtml,Player Value Leaderboards
1,WAR Position Players,/leaders/WAR_bat_active.shtml,Batting Leaderboards


In [12]:
def process_statistic(table):
    def process_row(row):
        res = []
        for td in row.find_all('td'):
            res.append(re.sub('\xa0', ' ', td.text.strip()))
        #print(res)
        return res
        pass
    
    columns_name = [i.text.strip() for i in table.find('thead').find_all('th')]
    #print(columns_name)
    
    data = []
    for row in table.find_all('tr')[1:]:
        #print(row)
        processed_row = process_row(row)
        if not processed_row:
            continue
        if processed_row[0] == '':
            processed_row[0] = data[-1][0]
        data.append(processed_row)
    
    df = pd.DataFrame(data, columns=columns_name)
    
    return df

In [24]:
folder = 'active'
all_active = []
for i in tqdm(range(min(df_active.shape[0], 20000))):
    stat_name = re.sub(' ', '_', df_active.loc[i, 'Statistic'])
    stat_name = re.sub('[\./]', '', stat_name)
    file_name = f'{i:03}_' + stat_name
    path_to_save = os.path.join(folder, file_name + '.csv')
    #print(path_to_save)
    
    content = get_parsed(baseball_base_url + df_active.loc[i, 'Active'])
    table = content.find('table')
    
    df = process_statistic(table)
    df['Rank'] = df['Rank'].map(lambda x: x[:-1]).astype(int)
    cat = df_active.loc[i, 'Category']
    cat = " ".join(cat.split(" ")[:-1])
    all_active.append([(cat, df_active.loc[i, 'Statistic']), df])

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [00:40<00:00,  4.47it/s]


In [45]:
with open('pkl_leaders_active.pkl', 'wb') as f:
    pickle.dump(all_active, f)
with open('pkl_leaders_active.pkl', 'rb') as f:
    all_active = pickle.load(f)

In [47]:
test_id = 43
print(all_active[test_id][0])
test_df = all_active[test_id][1]
test_df[test_df['Rank'].isin([1,2,3,4])]

('Batting', 'Championship WPA (cWPA)')


,Rank,"Player (yrs, age)",Championship WPA (cWPA),Bats
0,1,"Albert Pujols (20, 40)",48.61,R
1,2,"Miguel Cabrera (18, 37)",42.03,R
2,3,"Joey Votto (14, 36)",21.19,L
3,4,"Paul Goldschmidt (10, 32)",21.10,R


# Generate questions

In [40]:
with open('pkl_leaders_active.pkl', 'rb') as f:
    all_active = pickle.load(f)

In [57]:
wrong_count = 4
columns = ['Question', 'Correct'] + ['Wrong_' + str(i + 1) for i in range(wrong_count)] +  ['tags']
ranking_tag = 'Active Ranking'
leader_name = 'active'
question_list = []

import random
random.seed(1000)
for (cat, stat), df in tqdm(all_active[:min(len(all_active), 1000)]):
    name_column = df.columns[1]
    stat_column = df.columns[2]
    ranks_unique = df['Rank'].unique()
    
    small_range = 6
    middle_range = int(small_range * 1.8)
    large_range = int(middle_range * 1.8)
    small_range_rank = list(ranks_unique[:small_range])
    middle_range_rank = list(ranks_unique[:middle_range])
    large_range_rank = list(ranks_unique[:large_range])
    
    if len(ranks_unique) < 10:
        continue
    # who is number 1st - ith
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        question = f'Which of the following {player_or_manager}s is ranked #{rank} in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        # get one player that has the correct rank
        correct = df[df['Rank'] == (rank)][name_column].tolist()[0]
        
        # generate wrong answers
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        options = random.sample(middle_range_rank, k=min(subselect, len(middle_range_rank)))
        options = [option for option in options if option != rank][:wrong_count]
        wrongs = [df[df['Rank'] == (rank)][name_column].tolist()[0] for rank in options]
        
        # check whether the wrong answer is appropriate or not
        screen_flag = False
        for wrong in wrongs:
            # all the ranks of this player
            ranks_this_player = df[df[name_column].str.contains(clean_name(wrong))]['Rank'].tolist()
            if rank in ranks_this_player:
                screen_flag = True
                break
        if screen_flag:
            continue
        
        # if there is repeating names
        all_options = [clean_name(wrong) for wrong in [correct] + wrongs]
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + all_options + [tags])
        
        
    # what is the highest rank of the specific player in the leaderboard
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        player_name = df[df['Rank'] == (rank)][name_column].tolist()[0]
        question = f'What is the highest ranking of {clean_name(player_name)}' + \
            f' in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        
        # find the highest rank of this player
        highest_rank_this_player = min(df[df[name_column].str.contains(clean_name(player_name))]['Rank'].tolist())
        # correct rank
        correct = rank
        
        if correct != highest_rank_this_player:
            continue
        
        # wrong rank
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        options = random.sample(middle_range_rank, k=min(subselect, len(middle_range_rank)))
        options = [option for option in options if option != rank][:wrong_count]
        
        all_options = [correct] + options
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + all_options + [tags])
    
    # who is ranked highest of the following palyers
    for i in range(small_range):
        rank = small_range_rank[i]
        player_or_manager = 'manager' if 'Manager' in cat else 'player'
        player_name = df[df['Rank'] == (rank)][name_column].tolist()[0]
        question = f'Which of the following {player_or_manager}s ranks the highest' + \
            f' in the {leader_name} leaderboard for \'{stat} ({cat})\'?'
        correct = player_name
        
        # get the number of wrongs to select
        subselect = random.randrange(wrong_count + 1, wrong_count * 2)
        # select ranks that are bigger than rank
        options = random.sample(large_range_rank[(i + 1):], k=min(subselect, len(large_range_rank[(i + 1):])))
        options = [option for option in options if option != rank][:wrong_count]
        wrongs = [df[df['Rank'] == (rank)][name_column].tolist()[0] for rank in options]
        
        all_options = [clean_name(wrong) for wrong in [correct] + wrongs]
        
        # get highest for each player
        #print([correct] + wrongs)
        highest_each_player = [min(df[df[name_column].str.contains(clean_name(player_name))]['Rank'].tolist()) 
                               for player_name in all_options]
        zipped = list(zip(all_options, highest_each_player))
        zipped.sort(key=lambda x: x[1])
        if len(set(all_options)) != len(all_options):
            continue
            
        tags = [ranking_tag, player_or_manager]
        question_list.append([question] + [opt for opt, rk in zipped] + [tags])

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [00:10<00:00, 17.84it/s]


In [58]:
len(question_list)

3204

In [59]:
active_ranking_questions = pd.DataFrame(question_list, columns=columns)

In [62]:
with open('questions_active_ranking.pkl', 'wb') as f:
    pickle.dump(active_ranking_questions, f)
with open('questions_active_ranking.pkl', 'rb') as f:
    active_ranking_questions = pickle.load(f)

In [63]:
active_ranking_questions.iloc[-2614, :].tolist()

["Which of the following players ranks the highest in the active leaderboard for 'Intentional Bases on Balls (Batting)'?",
 'Joey Votto',
 'Robinson Cano',
 'Mike Trout',
 'Jay Bruce',
 'Matt Kemp',
 ['Active Ranking', 'player']]